<a href="https://colab.research.google.com/github/TADY25/Unil_Tissot/blob/main/DM_Tissot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U spacy
!python -m spacy download fr
!python -m spacy download fr_core_news_md

  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
⚠ As of spaCy v3.0, shortcuts like 'fr' are deprecated. Please use the
full pipeline package name 'fr_core_news_sm' instead.
     |████████████████████████████████| 17.4 MB 470 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
     |████████████████████████████████| 46.9 MB 120 kB/s 
✔ Download and installation successful
You can no

In [ ]:
# Import additional packages
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
import string
from spacy.lang.fr.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
df=pd.read_csv('training_data.csv')
#df_1=df["sentence"].astype(str)

df_1=df["sentence"]



In [ ]:
sp = spacy.load('fr_core_news_sm')
stopwords = spacy.lang.fr.stop_words.STOP_WORDS
punct=[",",";",".",":","!","?","\"","(",")","-"]

def spacy_tokenizer(sentence):
    mytokens = sp(sentence)
    mytokens = [ word.lemma_.lower().strip() for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punct ]

    return mytokens


In [ ]:

tfidf_vector = TfidfVectorizer(tokenizer=spacy_tokenizer)

In [ ]:
X = df_1
ylabels = df['difficulty'] # the labels, or answers, we want to test against

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2, stratify=ylabels)

X_train

4539    Elle me dit toujours qu'il faut lui parler de ...
3223    Coiffés de chapeaux en carton, habillés en vie...
3899     Illusion du laid qui est une opinion changeante!
1815    C'est pourquoi, si près de 50 % de nos essais ...
4318                                   J'ai une question.
                              ...                        
1094    Ici, tout est mêlé et accéléré , souligne la s...
3627    Il faut bien faire attention à ses vêtements e...
1924    Elle signifie "transmettre à quelqu'un un mess...
1545    Un passager peut se tenir à califourchon derri...
4126    Lorsqu'il eut reçu son premier salaire en tant...
Name: sentence, Length: 3840, dtype: object

In [ ]:


def classifier_set(classifier_type):
  if classifier_type == 0:
    classifier_name="LogisticRegression"
    classifier = LogisticRegression()
  elif classifier_type ==1:
    classifier_name="KNeighborsClassifier"
    classifier = KNeighborsClassifier()
  elif classifier_type == 2:
    classifier_name="DecisionTreeClassifier"
    classifier = DecisionTreeClassifier()
  elif classifier_type == 3:
    classifier_name="RandomForestClassifier"
    classifier = RandomForestClassifier()
  elif classifier_type == 4:
    classifier_name="AdaBoostClassifier"
    classifier = AdaBoostClassifier()
  
  pipe = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', classifier)])

  pipe.fit(X_train, y_train)
  return pipe

In [ ]:
classifier_set(1)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function spacy_tokenizer at 0x7fc85b5bef80>)),
                ('classifier', KNeighborsClassifier(n_neighbors=2))])

In [ ]:
# Evaluate the model
def evaluate(true, pred):
    precision = precision_score(true, pred, average='micro')
    recall = recall_score(true, pred, average='micro')
    f1 = f1_score(true, pred, average='micro')
    print(confusion_matrix(true, pred))
    print(accuracy_score(true, pred))
    print(precision, recall, f1)

In [ ]:
# Predictions
y_pred = pipe.predict(X_test)

# Evaluation - test set
evaluate(y_test, y_pred)



[[124  20   8   5   4   2]
 [ 24 111  13   3   5   3]
 [ 12  13 116   7   4   7]
 [  4   4   5 121  14  10]
 [  2   5   0   6 137  10]
 [  1   3   4   2   9 142]]
0.7822916666666667
0.7822916666666667 0.7822916666666667 0.7822916666666666


LOG REG
0.7822916666666667
0.7822916666666667 0.7822916666666667 0.7822916666666666
___________________________

DECISION TREE